# Temporal Forecasting 

### 1.1 Importing the necessary libraries and modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

### 1.2 Importing the necessary datasets

In [3]:
raw=pd.read_csv('../data/raw/ev_charging_patterns.csv')

In [4]:
processed_df=pd.read_csv('../data/processed/cleaned_charging_patterns.csv')

In [5]:
raw.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


In [6]:
processed_df.head()

,user_id,battery_capacity_(kwh),charging_station_id,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),charging_cost_(usd),time_of_day,day_of_week,state_of_charge_(start_%),...,charging_station_location_San Francisco,user_type_Casual Driver,user_type_Commuter,user_type_Long-Distance Traveler,start_hour,start_day,start_month,end_hour,end_day,end_month
0,1,1.645480,391,0.827415,-1.582085,0.765064,-0.880549,2,2,-0.821044,...,0,0,1,0,5,8,4,6,8,4
1,2,1.235035,428,-1.387924,0.814865,0.346684,-0.132395,0,1,-1.621201,...,1,1,0,0,8,8,4,14,8,4
2,3,0.022567,181,-1.076981,0.172798,0.114901,1.220378,0,4,-1.756717,...,1,0,1,0,11,8,4,18,8,4
3,4,-1.189901,327,1.685900,-0.945610,0.508216,-0.885339,2,6,1.412424,...,0,0,0,1,14,8,4,23,8,4
4,5,-1.189901,108,-1.054072,-0.235342,-1.152220,-1.152824,0,6,0.213128,...,0,0,0,1,16,8,4,20,8,4


In [7]:
processed_df.columns

Index(['user_id', 'battery_capacity_(kwh)', 'charging_station_id',
       'energy_consumed_(kwh)', 'charging_duration_(hours)',
       'charging_rate_(kw)', 'charging_cost_(usd)', 'time_of_day',
       'day_of_week', 'state_of_charge_(start_%)', 'state_of_charge_(end_%)',
       'distance_driven_(since_last_charge)_(km)', 'temperature_(°c)',
       'vehicle_age_(years)', 'charger_type', 'vehicle_model_BMW i3',
       'vehicle_model_Chevy Bolt', 'vehicle_model_Hyundai Kona',
       'vehicle_model_Nissan Leaf', 'vehicle_model_Tesla Model 3',
       'charging_station_location_Chicago',
       'charging_station_location_Houston',
       'charging_station_location_Los Angeles',
       'charging_station_location_New York',
       'charging_station_location_San Francisco', 'user_type_Casual Driver',
       'user_type_Commuter', 'user_type_Long-Distance Traveler', 'start_hour',
       'start_day', 'start_month', 'end_hour', 'end_day', 'end_month'],
      dtype='object')

In [8]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   User ID                                   1320 non-null   object 
 1   Vehicle Model                             1320 non-null   object 
 2   Battery Capacity (kWh)                    1320 non-null   float64
 3   Charging Station ID                       1320 non-null   object 
 4   Charging Station Location                 1320 non-null   object 
 5   Charging Start Time                       1320 non-null   object 
 6   Charging End Time                         1320 non-null   object 
 7   Energy Consumed (kWh)                     1254 non-null   float64
 8   Charging Duration (hours)                 1320 non-null   float64
 9   Charging Rate (kW)                        1254 non-null   float64
 10  Charging Cost (USD)                 

In [9]:
raw.describe()

,Battery Capacity (kWh),Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years)
count,1320.000000,1254.000000,1320.000000,1254.000000,1320.000000,1320.000000,1320.000000,1254.000000,1320.000000,1320.000000
mean,74.534692,42.642894,2.269377,25.963003,22.551352,49.130012,75.141590,153.596788,15.263591,3.612843
std,20.626914,22.411705,1.061037,14.011326,10.751494,24.074134,17.080580,86.004987,14.831216,2.309824
min,1.532807,0.045772,0.095314,1.472549,0.234317,2.325959,7.604224,0.862361,-10.724770,0.000000
25%,62.000000,23.881193,1.397623,13.856583,13.368141,27.786903,62.053266,79.445335,2.800664,2.000000
50%,75.000000,42.691405,2.258136,25.603799,22.076360,48.241771,75.682496,152.259867,14.630846,4.000000
75%,85.000000,61.206218,3.112806,37.502998,31.646044,69.277921,88.201370,226.073284,27.981810,6.000000
max,193.003074,152.238758,7.635145,97.342255,69.407743,152.489761,177.708666,398.364775,73.169588,11.688592


In [10]:
raw.isnull().sum()

User ID                                      0
Vehicle Model                                0
Battery Capacity (kWh)                       0
Charging Station ID                          0
Charging Station Location                    0
Charging Start Time                          0
Charging End Time                            0
Energy Consumed (kWh)                       66
Charging Duration (hours)                    0
Charging Rate (kW)                          66
Charging Cost (USD)                          0
Time of Day                                  0
Day of Week                                  0
State of Charge (Start %)                    0
State of Charge (End %)                      0
Distance Driven (since last charge) (km)    66
Temperature (°C)                             0
Vehicle Age (years)                          0
Charger Type                                 0
User Type                                    0
dtype: int64

### 1.3 MAking the column names compact

In [11]:
raw.columns = raw.columns.str.lower().str.replace(' ', '_')
raw.columns

Index(['user_id', 'vehicle_model', 'battery_capacity_(kwh)',
       'charging_station_id', 'charging_station_location',
       'charging_start_time', 'charging_end_time', 'energy_consumed_(kwh)',
       'charging_duration_(hours)', 'charging_rate_(kw)',
       'charging_cost_(usd)', 'time_of_day', 'day_of_week',
       'state_of_charge_(start_%)', 'state_of_charge_(end_%)',
       'distance_driven_(since_last_charge)_(km)', 'temperature_(°c)',
       'vehicle_age_(years)', 'charger_type', 'user_type'],
      dtype='object')

In [12]:
raw.dtypes

user_id                                      object
vehicle_model                                object
battery_capacity_(kwh)                      float64
charging_station_id                          object
charging_station_location                    object
charging_start_time                          object
charging_end_time                            object
energy_consumed_(kwh)                       float64
charging_duration_(hours)                   float64
charging_rate_(kw)                          float64
charging_cost_(usd)                         float64
time_of_day                                  object
day_of_week                                  object
state_of_charge_(start_%)                   float64
state_of_charge_(end_%)                     float64
distance_driven_(since_last_charge)_(km)    float64
temperature_(°c)                            float64
vehicle_age_(years)                         float64
charger_type                                 object
user_type   

In [13]:
raw.head()

,user_id,vehicle_model,battery_capacity_(kwh),charging_station_id,charging_station_location,charging_start_time,charging_end_time,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),charging_cost_(usd),time_of_day,day_of_week,state_of_charge_(start_%),state_of_charge_(end_%),distance_driven_(since_last_charge)_(km),temperature_(°c),vehicle_age_(years),charger_type,user_type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


### 1.4 Converting into the date-time format

In [14]:
# Convert to datetime format
raw['charging_start_time'] = pd.to_datetime(raw['charging_start_time'])
raw['charging_end_time'] = pd.to_datetime(raw['charging_end_time'])

### 1.5 Extracting day, hour, month, year, weekdays kind of information

In [15]:
raw['start_hour'] = raw['charging_start_time'].dt.hour
raw['start_day'] = raw['charging_start_time'].dt.day
raw['start_month'] = raw['charging_start_time'].dt.month
raw['start_year'] = raw['charging_start_time'].dt.year
raw['weekday'] = raw['charging_start_time'].dt.weekday
raw['is_weekend'] = raw['weekday'].isin([5, 6]).astype(int) 

### 2. Gap Interval (Time since last charge)

### 2.1 Gap interval (time_since_last_charge) tells the model how frequently a user charges.

In [16]:
raw['time_since_last_charge'] = raw['charging_start_time'].diff().dt.total_seconds() / 3600

### 2.2 Generating Lags from previous info

A lag in time-series analysis refers to the use of past data points to predict future values.

Energy consumed (lag_1): The energy consumed during the previous charging session.

Energy consumed (lag_2): The energy consumed two charging sessions ago.

These lag features are useful because they give the model information about past behavior that might help it predict future behavior (e.g., a higher energy consumption this session could be influenced by a higher consumption last session).

Why Lags are Important:

    * Capturing Temporal Dependencies: In time series forecasting, past behavior often influences future values, making lags essential.

    * Trends & Patterns: Lags can help capture recurring patterns (e.g., high energy consumption during specific times of the day).

In [17]:
raw['energy_consumed_lag_1'] = raw['energy_consumed_(kwh)'].shift(1)
raw['energy_consumed_lag_2'] = raw['energy_consumed_(kwh)'].shift(2)
raw['charging_duration_lag_1'] = raw['charging_duration_(hours)'].shift(1)
raw['charging_duration_lag_2'] = raw['charging_duration_(hours)'].shift(2)

### 2.3 Generating Rolling Statistics 

A rolling window is a way to perform calculations across a subset of consecutive data points (in this case, a window size of 3) and then shift this window one data point at a time.

Window Size = 3 means that, for each data point, you calculate the mean or standard deviation based on the current data point and the two previous ones (total of 3 points).

The window slides over time, so the calculations for the next data point will use the next 3 data points.

Why Use Rolling Statistics?

Smoothing: Rolling statistics (mean, std, etc.) help smooth out fluctuations and capture trends in the data.

Trend Analysis: It allows the model to learn from the recent behavior over a specific period (like the last 3 data points).

Noise Reduction: Rolling means reduce noise and outliers, making the data less volatile.

In [18]:
raw['energy_consumed_roll_mean_3'] = raw['energy_consumed_(kwh)'].rolling(window=3).mean()
raw['energy_consumed_roll_std_3'] = raw['energy_consumed_(kwh)'].rolling(window=3).std()
raw['charging_duration_roll_mean_3'] = raw['charging_duration_(hours)'].rolling(window=3).mean()
raw['charging_duration_roll_std_3'] = raw['charging_duration_(hours)'].rolling(window=3).std()

In [19]:
raw.isnull().sum()

user_id                                       0
vehicle_model                                 0
battery_capacity_(kwh)                        0
charging_station_id                           0
charging_station_location                     0
charging_start_time                           0
charging_end_time                             0
energy_consumed_(kwh)                        66
charging_duration_(hours)                     0
charging_rate_(kw)                           66
charging_cost_(usd)                           0
time_of_day                                   0
day_of_week                                   0
state_of_charge_(start_%)                     0
state_of_charge_(end_%)                       0
distance_driven_(since_last_charge)_(km)     66
temperature_(°c)                              0
vehicle_age_(years)                           0
charger_type                                  0
user_type                                     0
start_hour                              

### 3. Extracting all the new columns and forming a new dataset from it

In [20]:
temporal_features = raw[['weekday','is_weekend','time_since_last_charge','energy_consumed_lag_1', 'energy_consumed_lag_2', 'charging_duration_lag_1','charging_duration_lag_2', 'energy_consumed_roll_mean_3', 
                        'energy_consumed_roll_std_3', 'charging_duration_roll_mean_3', 'charging_duration_roll_std_3']]

In [21]:
temporal_features.dtypes

weekday                            int32
is_weekend                         int64
time_since_last_charge           float64
energy_consumed_lag_1            float64
energy_consumed_lag_2            float64
charging_duration_lag_1          float64
charging_duration_lag_2          float64
energy_consumed_roll_mean_3      float64
energy_consumed_roll_std_3       float64
charging_duration_roll_mean_3    float64
charging_duration_roll_std_3     float64
dtype: object

In [22]:
temporal_features

,weekday,is_weekend,time_since_last_charge,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1.0,60.712346,NaN,0.591363,NaN,NaN,NaN,NaN,NaN
2,0,0,1.0,12.339275,60.712346,3.133652,0.591363,30.726832,26.189176,2.059223,1.316016
3,0,0,1.0,19.128876,12.339275,2.452653,3.133652,36.975325,36.947215,2.284245,0.944934
4,0,0,1.0,79.457824,19.128876,1.266431,2.452653,39.405268,34.687433,1.912950,0.600281
...,...,...,...,...,...,...,...,...,...,...,...
1315,5,1,1.0,43.251453,60.725144,2.501809,2.365991,48.662750,10.464716,2.098081,0.585606
1316,5,1,1.0,42.011654,43.251453,1.426444,2.501809,51.149653,14.766799,2.388822,0.911153
1317,5,1,1.0,68.185853,42.011654,3.238212,1.426444,43.030869,24.661177,2.643926,1.054469
1318,5,1,1.0,18.895102,68.185853,3.267122,3.238212,33.612402,30.051532,3.086621,0.287964


In [23]:
temporal_features.isnull().sum()

weekday                            0
is_weekend                         0
time_since_last_charge             1
energy_consumed_lag_1             67
energy_consumed_lag_2             68
charging_duration_lag_1            1
charging_duration_lag_2            2
energy_consumed_roll_mean_3      194
energy_consumed_roll_std_3       194
charging_duration_roll_mean_3      2
charging_duration_roll_std_3       2
dtype: int64

In [24]:
temporal_features['time_since_last_charge']=temporal_features['time_since_last_charge'].fillna(temporal_features['time_since_last_charge'].median())
temporal_features['charging_duration_lag_1']=temporal_features['charging_duration_lag_1'].fillna(temporal_features['charging_duration_lag_1'].median())
temporal_features['charging_duration_lag_2']=temporal_features['charging_duration_lag_2'].fillna(temporal_features['charging_duration_lag_2'].median())
temporal_features['energy_consumed_lag_1']=temporal_features['energy_consumed_lag_1'].fillna(temporal_features['energy_consumed_lag_1'].median())
temporal_features['energy_consumed_lag_2']=temporal_features['energy_consumed_lag_2'].fillna(temporal_features['energy_consumed_lag_2'].median())
temporal_features['charging_duration_roll_mean_3']=temporal_features['charging_duration_roll_mean_3'].fillna(temporal_features['charging_duration_roll_mean_3'].median())
temporal_features['charging_duration_roll_std_3']=temporal_features['charging_duration_roll_std_3'].fillna(temporal_features['charging_duration_roll_std_3'].median())
temporal_features['energy_consumed_roll_mean_3']=temporal_features['energy_consumed_roll_mean_3'].fillna(temporal_features['energy_consumed_roll_mean_3'].median())
temporal_features['energy_consumed_roll_std_3']=temporal_features['energy_consumed_roll_std_3'].fillna(temporal_features['energy_consumed_roll_std_3'].median()) 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20192\3264615553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_features['time_since_last_charge']=temporal_features['time_since_last_charge'].fillna(temporal_features['time_since_last_charge'].median())
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20192\3264615553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temporal_features['charging_duration_lag_1']=temporal_features['charging_duration_lag_1'].fillna(temporal_features['charging_duration_lag_1

In [25]:
temporal_features.isnull().sum()

weekday                          0
is_weekend                       0
time_since_last_charge           0
energy_consumed_lag_1            0
energy_consumed_lag_2            0
charging_duration_lag_1          0
charging_duration_lag_2          0
energy_consumed_roll_mean_3      0
energy_consumed_roll_std_3       0
charging_duration_roll_mean_3    0
charging_duration_roll_std_3     0
dtype: int64

In [26]:
temporal_features.head()

,weekday,is_weekend,time_since_last_charge,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,0,0,1.0,42.638018,42.691405,2.255525,2.254276,42.830564,19.799001,2.262460,0.923453
1,0,0,1.0,60.712346,42.691405,0.591363,2.254276,42.830564,19.799001,2.262460,0.923453
2,0,0,1.0,12.339275,60.712346,3.133652,0.591363,30.726832,26.189176,2.059223,1.316016
3,0,0,1.0,19.128876,12.339275,2.452653,3.133652,36.975325,36.947215,2.284245,0.944934
4,0,0,1.0,79.457824,19.128876,1.266431,2.452653,39.405268,34.687433,1.912950,0.600281


In [27]:
temporal_features['weekday'].unique()

array([0, 1, 2, 3, 4, 5, 6], dtype=int32)

In [28]:
temporal_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   weekday                        1320 non-null   int32  
 1   is_weekend                     1320 non-null   int64  
 2   time_since_last_charge         1320 non-null   float64
 3   energy_consumed_lag_1          1320 non-null   float64
 4   energy_consumed_lag_2          1320 non-null   float64
 5   charging_duration_lag_1        1320 non-null   float64
 6   charging_duration_lag_2        1320 non-null   float64
 7   energy_consumed_roll_mean_3    1320 non-null   float64
 8   energy_consumed_roll_std_3     1320 non-null   float64
 9   charging_duration_roll_mean_3  1320 non-null   float64
 10  charging_duration_roll_std_3   1320 non-null   float64
dtypes: float64(9), int32(1), int64(1)
memory usage: 108.4 KB


### 4. Scaling the dataset

In [29]:
scaler = StandardScaler()
temporal_scaled= scaler.fit_transform(temporal_features)

In [30]:
temporal_df = pd.DataFrame(temporal_scaled, columns=['weekday','weekend','time_since_last_charge','energy_consumed_lag_1','energy_consumed_lag_2', 'charging_duration_lag_1', 
                                                    'charging_duration_lag_2', 'energy_consumed_roll_mean_3', 'energy_consumed_roll_std_3', 
                                                    'charging_duration_roll_mean_3', 'charging_duration_roll_std_3'], index=raw.index)


In [31]:
temporal_df.head()

,weekday,weekend,time_since_last_charge,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,-1.490321,-0.612372,0.0,0.000517,0.001836,-0.012008,-0.012829,0.031932,-0.015945,-0.011086,-0.049419
1,-1.490321,-0.612372,0.0,0.828557,0.001836,-1.582178,-0.012829,0.031932,-0.015945,-0.011086,-0.049419
2,-1.490321,-0.612372,0.0,-1.387560,0.827978,0.816522,-1.581946,-0.960906,0.708437,-0.348877,0.746620
3,-1.490321,-0.612372,0.0,-1.076508,-1.389610,0.173986,0.816946,-0.448358,1.927954,0.025122,-0.005860
4,-1.490321,-0.612372,0.0,1.687344,-1.078352,-0.945239,0.174358,-0.249036,1.671788,-0.591991,-0.704745


In [32]:
temporal_df.isnull().sum()

weekday                          0
weekend                          0
time_since_last_charge           0
energy_consumed_lag_1            0
energy_consumed_lag_2            0
charging_duration_lag_1          0
charging_duration_lag_2          0
energy_consumed_roll_mean_3      0
energy_consumed_roll_std_3       0
charging_duration_roll_mean_3    0
charging_duration_roll_std_3     0
dtype: int64

### 5 Concating the new dataset with processed dataset

In [33]:
processed_df = pd.concat([processed_df, temporal_df], axis=1)

In [34]:
processed_df.head()

,user_id,battery_capacity_(kwh),charging_station_id,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),charging_cost_(usd),time_of_day,day_of_week,state_of_charge_(start_%),...,weekend,time_since_last_charge,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,1,1.645480,391,0.827415,-1.582085,0.765064,-0.880549,2,2,-0.821044,...,-0.612372,0.0,0.000517,0.001836,-0.012008,-0.012829,0.031932,-0.015945,-0.011086,-0.049419
1,2,1.235035,428,-1.387924,0.814865,0.346684,-0.132395,0,1,-1.621201,...,-0.612372,0.0,0.828557,0.001836,-1.582178,-0.012829,0.031932,-0.015945,-0.011086,-0.049419
2,3,0.022567,181,-1.076981,0.172798,0.114901,1.220378,0,4,-1.756717,...,-0.612372,0.0,-1.387560,0.827978,0.816522,-1.581946,-0.960906,0.708437,-0.348877,0.746620
3,4,-1.189901,327,1.685900,-0.945610,0.508216,-0.885339,2,6,1.412424,...,-0.612372,0.0,-1.076508,-1.389610,0.173986,0.816946,-0.448358,1.927954,0.025122,-0.005860
4,5,-1.189901,108,-1.054072,-0.235342,-1.152220,-1.152824,0,6,0.213128,...,-0.612372,0.0,1.687344,-1.078352,-0.945239,0.174358,-0.249036,1.671788,-0.591991,-0.704745


In [35]:
processed_df.columns

Index(['user_id', 'battery_capacity_(kwh)', 'charging_station_id',
       'energy_consumed_(kwh)', 'charging_duration_(hours)',
       'charging_rate_(kw)', 'charging_cost_(usd)', 'time_of_day',
       'day_of_week', 'state_of_charge_(start_%)', 'state_of_charge_(end_%)',
       'distance_driven_(since_last_charge)_(km)', 'temperature_(°c)',
       'vehicle_age_(years)', 'charger_type', 'vehicle_model_BMW i3',
       'vehicle_model_Chevy Bolt', 'vehicle_model_Hyundai Kona',
       'vehicle_model_Nissan Leaf', 'vehicle_model_Tesla Model 3',
       'charging_station_location_Chicago',
       'charging_station_location_Houston',
       'charging_station_location_Los Angeles',
       'charging_station_location_New York',
       'charging_station_location_San Francisco', 'user_type_Casual Driver',
       'user_type_Commuter', 'user_type_Long-Distance Traveler', 'start_hour',
       'start_day', 'start_month', 'end_hour', 'end_day', 'end_month',
       'weekday', 'weekend', 'time_since_last_

In [36]:
processed_df.shape

(1320, 45)

In [37]:
processed_df.columns=processed_df.columns.str.replace('vehicle_model_BMW i3','vehicle_model_bmw_i3')
processed_df.columns=processed_df.columns.str.replace('vehicle_model_Chevy Bolt','vehicle_model_chevy_bolt')
processed_df.columns=processed_df.columns.str.replace('vehicle_model_Hyundai Kona','vehicle_model_hyundai_kona')
processed_df.columns=processed_df.columns.str.replace('vehicle_model_Nissan Leaf','vehicle_model_nissan_leaf')
processed_df.columns=processed_df.columns.str.replace('vehicle_model_Tesla Model 3','vehicle_model_tesla_model3')
processed_df.columns=processed_df.columns.str.replace('charging_station_location_Chicago','charg_stn_loc_chicago')
processed_df.columns=processed_df.columns.str.replace('charging_station_location_Houston','charg_stn_loc_houston')
processed_df.columns=processed_df.columns.str.replace('charging_station_location_Los Angeles','charg_stn_loc_LA')
processed_df.columns=processed_df.columns.str.replace('charging_station_location_New York','charg_stn_loc_newyork')
processed_df.columns=processed_df.columns.str.replace('charging_station_location_San Francisco','charg_stn_loc_san_fran')
processed_df.columns=processed_df.columns.str.replace('temperature_(°c)', 'temp(°c)')

In [38]:
processed_df.isnull().sum().sum()

np.int64(0)

In [39]:
processed_df.head(20)

,user_id,battery_capacity_(kwh),charging_station_id,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),charging_cost_(usd),time_of_day,day_of_week,state_of_charge_(start_%),...,weekend,time_since_last_charge,energy_consumed_lag_1,energy_consumed_lag_2,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3
0,1,1.645480,391,0.827415,-1.582085,0.765064,-0.880549,2,2,-0.821044,...,-0.612372,0.0,0.000517,0.001836,-0.012008,-0.012829,0.031932,-0.015945,-0.011086,-0.049419
1,2,1.235035,428,-1.387924,0.814865,0.346684,-0.132395,0,1,-1.621201,...,-0.612372,0.0,0.828557,0.001836,-1.582178,-0.012829,0.031932,-0.015945,-0.011086,-0.049419
2,3,0.022567,181,-1.076981,0.172798,0.114901,1.220378,0,4,-1.756717,...,-0.612372,0.0,-1.387560,0.827978,0.816522,-1.581946,-0.960906,0.708437,-0.348877,0.746620
3,4,-1.189901,327,1.685900,-0.945610,0.508216,-0.885339,2,6,1.412424,...,-0.612372,0.0,-1.076508,-1.389610,0.173986,0.816946,-0.448358,1.927954,0.025122,-0.005860
4,5,-1.189901,108,-1.054072,-0.235342,-1.152220,-1.152824,0,6,0.213128,...,-0.612372,0.0,1.687344,-1.078352,-0.945239,0.174358,-0.249036,1.671788,-0.591991,-0.704745
5,6,-1.189901,335,0.024539,-1.038753,-0.850505,1.335110,2,6,1.084052,...,-0.612372,0.0,-1.053591,1.687335,-0.234452,-0.944956,0.408613,1.156189,-1.303909,-0.976877
6,7,0.507554,162,-0.264852,1.197624,0.017591,-0.031368,2,5,0.482932,...,-0.612372,0.0,0.025399,-1.055419,-1.038450,-0.234113,-0.756059,-0.878469,-0.044525,0.514509
7,8,0.022567,302,0.404034,0.363951,0.055516,-1.186753,1,1,0.293858,...,-0.612372,0.0,-0.264093,0.024287,1.199561,-1.038174,0.114485,-1.430005,0.307626,0.508764
8,9,-0.607917,493,0.043372,-0.514007,-0.853406,0.922422,2,3,-0.650896,...,-0.612372,0.0,0.405028,-0.265398,0.365279,1.200015,0.125729,-1.431657,0.615971,-0.081141
9,10,-1.189901,452,1.658916,-0.228639,-1.039023,-0.115288,0,3,-0.902994,...,-0.612372,0.0,0.044239,0.404167,-0.513321,0.365666,1.274290,-0.161286,-0.222114,-0.958784


### 6. Cyclic Time Encoding

Why this matters for EV charging ?

EV charging behavior:

* Peaks in morning & evening
* Drops at night
* Different on weekends vs weekdays

Cyclic encoding lets models learn:

* Rush-hour charging
* Weekend patterns
* Smooth transitions between time slots

I applied cyclic encoding because:
* temporal features are periodic, and sine–cosine transformation preserves proximity at boundaries like midnight and week transitions

In [40]:
processed_df['start_hour_sin'] = np.sin(2 * np.pi * processed_df['start_hour'] / 24)
processed_df['start_hour_cos'] = np.cos(2 * np.pi * processed_df['start_hour'] / 24)

processed_df['weekday_sin'] = np.sin(2 * np.pi * processed_df['weekday'] / 7)
processed_df['weekday_cos'] = np.cos(2 * np.pi * processed_df['weekday'] / 7)

In [41]:
processed_df.head()

,user_id,battery_capacity_(kwh),charging_station_id,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),charging_cost_(usd),time_of_day,day_of_week,state_of_charge_(start_%),...,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3,start_hour_sin,start_hour_cos,weekday_sin,weekday_cos
0,1,1.645480,391,0.827415,-1.582085,0.765064,-0.880549,2,2,-0.821044,...,-0.012008,-0.012829,0.031932,-0.015945,-0.011086,-0.049419,0.965926,0.258819,-0.972958,0.230983
1,2,1.235035,428,-1.387924,0.814865,0.346684,-0.132395,0,1,-1.621201,...,-1.582178,-0.012829,0.031932,-0.015945,-0.011086,-0.049419,0.866025,-0.500000,-0.972958,0.230983
2,3,0.022567,181,-1.076981,0.172798,0.114901,1.220378,0,4,-1.756717,...,0.816522,-1.581946,-0.960906,0.708437,-0.348877,0.746620,0.258819,-0.965926,-0.972958,0.230983
3,4,-1.189901,327,1.685900,-0.945610,0.508216,-0.885339,2,6,1.412424,...,0.173986,0.816946,-0.448358,1.927954,0.025122,-0.005860,-0.500000,-0.866025,-0.972958,0.230983
4,5,-1.189901,108,-1.054072,-0.235342,-1.152220,-1.152824,0,6,0.213128,...,-0.945239,0.174358,-0.249036,1.671788,-0.591991,-0.704745,-0.866025,-0.500000,-0.972958,0.230983


In [42]:
processed_df.shape

(1320, 49)

In [43]:
new_cols=processed_df[['start_hour_sin' , 'start_hour_cos', 'weekday_sin' , 'weekday_cos' , 'time_since_last_charge']]

In [44]:
scaler = StandardScaler()
scaled= scaler.fit_transform(new_cols)

In [45]:
processed_df.dtypes

user_id                                       int64
battery_capacity_(kwh)                      float64
charging_station_id                           int64
energy_consumed_(kwh)                       float64
charging_duration_(hours)                   float64
charging_rate_(kw)                          float64
charging_cost_(usd)                         float64
time_of_day                                   int64
day_of_week                                   int64
state_of_charge_(start_%)                   float64
state_of_charge_(end_%)                     float64
distance_driven_(since_last_charge)_(km)    float64
temp(°c)                                    float64
vehicle_age_(years)                         float64
charger_type                                  int64
vehicle_model_bmw_i3                          int64
vehicle_model_chevy_bolt                      int64
vehicle_model_hyundai_kona                    int64
vehicle_model_nissan_leaf                     int64
vehicle_mode

In [46]:
processed_df.isnull().sum()

user_id                                     0
battery_capacity_(kwh)                      0
charging_station_id                         0
energy_consumed_(kwh)                       0
charging_duration_(hours)                   0
charging_rate_(kw)                          0
charging_cost_(usd)                         0
time_of_day                                 0
day_of_week                                 0
state_of_charge_(start_%)                   0
state_of_charge_(end_%)                     0
distance_driven_(since_last_charge)_(km)    0
temp(°c)                                    0
vehicle_age_(years)                         0
charger_type                                0
vehicle_model_bmw_i3                        0
vehicle_model_chevy_bolt                    0
vehicle_model_hyundai_kona                  0
vehicle_model_nissan_leaf                   0
vehicle_model_tesla_model3                  0
charg_stn_loc_chicago                       0
charg_stn_loc_houston             

In [47]:
processed_df.head()

,user_id,battery_capacity_(kwh),charging_station_id,energy_consumed_(kwh),charging_duration_(hours),charging_rate_(kw),charging_cost_(usd),time_of_day,day_of_week,state_of_charge_(start_%),...,charging_duration_lag_1,charging_duration_lag_2,energy_consumed_roll_mean_3,energy_consumed_roll_std_3,charging_duration_roll_mean_3,charging_duration_roll_std_3,start_hour_sin,start_hour_cos,weekday_sin,weekday_cos
0,1,1.645480,391,0.827415,-1.582085,0.765064,-0.880549,2,2,-0.821044,...,-0.012008,-0.012829,0.031932,-0.015945,-0.011086,-0.049419,0.965926,0.258819,-0.972958,0.230983
1,2,1.235035,428,-1.387924,0.814865,0.346684,-0.132395,0,1,-1.621201,...,-1.582178,-0.012829,0.031932,-0.015945,-0.011086,-0.049419,0.866025,-0.500000,-0.972958,0.230983
2,3,0.022567,181,-1.076981,0.172798,0.114901,1.220378,0,4,-1.756717,...,0.816522,-1.581946,-0.960906,0.708437,-0.348877,0.746620,0.258819,-0.965926,-0.972958,0.230983
3,4,-1.189901,327,1.685900,-0.945610,0.508216,-0.885339,2,6,1.412424,...,0.173986,0.816946,-0.448358,1.927954,0.025122,-0.005860,-0.500000,-0.866025,-0.972958,0.230983
4,5,-1.189901,108,-1.054072,-0.235342,-1.152220,-1.152824,0,6,0.213128,...,-0.945239,0.174358,-0.249036,1.671788,-0.591991,-0.704745,-0.866025,-0.500000,-0.972958,0.230983


### 7. Saving the Dataset

In [48]:
processed_df.to_csv("../data/processed/temporal_forecast_result.csv", index=False)